# MSc HI DTP Module: Week 4 workshop

## Summarising, Analysing and Communicating Data
In the lectures for this week, we have explored two (very) broad types of research question that we usually interested in: (1) association/prediction or (2) causal inference, and we have covered how these can be used to turn data into evidence. 

In this workshop, we will explore a real-world dataset to explore these concepts from the perspective of understanding outputs from different statistical methods. In the context of this module, the analytical methods are key in addressing the problem statement and monitoring /predicting the performance of the digital intervention.

We will extend the ideas covered in the lectures around using regression analysis to investigate associations within a dataset. We will compare the interpretation of this type of statistical study, with that which would be required to reach causal interpretation. 

Similar to the workshop from week 3, we will be using R within the Jupyter notebook envrioment for this activity. As before, having a detailed understanding of the R code is not the aim of this activity! Our primary goal is to examine how the general principles of the design cycle can be used to decide on when different types of statistical analyses would be appropriate in different digital transformation projects. 

### Notes about the software used in this workshop (repeat from week 3)
You will notice that this session is running using Jupyter notebooks. While, in practice, we might use R directly for analysis, we make use of Jupyter notebooks here since this allows you to run the analysis without having to install any software on your local machines.

You will be guided through the code syntax at each stage, to allow you to start to become familiar with how to change the code according to your own problems/research questions. Again, this workshop is not intended as a comprehensive guide to coding and statistical analysis. To reiterate, you could easily skip over the meaning of the code if you wish: focus instead on the what inference we can draw from different types of analysis, and therefore which ones you might design into your DTP case studies (and future projects).

Please refer to the short tutorial video on Moodle for how to use Jupyter notebooks. In short, you will see a mixture of text and R code, with the later included inside grey 'cells' - we call these code cells. When you click inside a code cell, you will be able to run the associated code using the "Run" box in the menu bar at the top of the screen (alternatively select from the menus: Cell -> Run Cells). 

__IMPORTANT: As you work through this workshop, run each code cell in the order they appear: you will also have space to write your own code if you wish.__

## "Types" of Statistical Analysis

As covered in the lectures, there are three broad “types” of common research questions when analysing data: 

1) Description

2) Causation

3) Association/ Prediction

The type of statistical analysis largely depends on the underlying research question (i.e. stage i of our design cycle: problem), the design of the study (i.e. stage ii of our design cycle: planning) and the type of data/outcomes we plan to analyse (i.e. stage iii of our design cycle: data).

In this workshop, we will focus on association, and we will explore the differences of this type of study with that aiming to investigate causation. We covered description in week 3.

## The Data

We will again be using the "Framingham" data. Please refer to week 3 material for an overview and description of these data.

## Load the data

To begin exploring these data, we first need to load it into the session. The data are stored in csv format, and the R code to load the data is as follows (noting, for the last time, that there is no expectation for you to delve deep into R coding in this activity):

In [ ]:
frmgham_data <- read.csv("frmgham2.csv", header = TRUE)

Recall that this means that we have loaded the data, and we can analyse the data by applying R functions to the frmgham_data variable. 

As previous, it is always a good idea to explore the first few lines of data to check that the data has been loaded correctly:

In [ ]:
head(frmgham_data)

## Exploring Associations using Regression
The ideas covered last week in terms of data summary and graphical exploration can take the analysis a suprisingly long way in terms of exploring single variables or visualising associations between pairs of variables (e.g. using a scatter plot to look at the correlation between two continuous variables). 

However, in most practical settings we are interested in predicting or explaining one variable (usually called the outcome), in relation to multiple other variables (sometimes called covariates or predictors). For example, we might be interested in predicting the risk of mortality following myocardial infarction, given a set of physiological parameters collected from a implantable device (one of the case studies in this module!). Alternatively, we might be interested in exploring what collection of variables are associated with developing cancer. In both of these situations, we might use statistical regression modelling to formally describe, in a mathematical way, the associations for the purposes of either prediction or explanation.

## Framingham case study
To motivate this, lets consider a "problem" and "plan" (from the design cycle) that we might consider for the Framingham dataset: lets imagine that we want to investigate which variables, collected at baseline, associate with systolic blood pressure (SBP). Our plan, therefore, is to use the baseline measurements from the dataset, where we SBP is our outcome and we intend to use all other variables that are collected in Framingham. 

Note, if we were designing this study from scratch and for a real project, we might wish to link with other data sources as well: think back to the data landscape from week 3 of this module. For example, we might wish to consider the index of multiple deprivation (IMD), or we might design a prospective study that incorperates wearable device data to investigate if activity levels associate with SBP. This is the benefit of taking a problem-centric approach to study design: careful thought at design stage can pay dividends in terms of inference and communication of results. 

Nonetheless, lets return to our hypothetical example. Our plan was to use the baseline measurements from the dataset, so lets start by subsetting/filtering the data (recall we saw how to do this in the last workshop):

In [ ]:
frmgham_1visit_data <- frmgham_data[!duplicated(frmgham_data$RANDID),] #restrict to first visits

Taking forward our learning from week 3, we might start the exploration by creating summarises/graphs to help us visualise this. For example, lets plot how BMI correlates with SBP, as a starting point:

In [ ]:
plot(x = frmgham_1visit_data$BMI, y = frmgham_1visit_data$SYSBP, xlab = "BMI", ylab = "SBP")

As we can see from the plot, visually there seems to be a slight positive correlation here: as BMI increases, so to does SBP. We will explore this formally within a linear regression model in a moment.

__Activity: Use the empty code cell below to perform other (relevant) descriptive statistics or plots in relation to our problem of interest.__

In [ ]:
##You can write R code here, then click the "Run" button in the toolbar at the top to execute your code.

Lets think back to the "problem" for our hypothetical example: we want to see what variables associate with SBP. We have already seen the there was an approximate association between BMI and SBP - but in terms of turning this into a ‘business-need' it is not particularly illuminating. 

This is where the idea of regression comes in. Whenever, we are interested in exploring how different variables interact or associate with each other, then regression is normally the first port-of-call in terms of statistical design. There are a large number of different types of regression analysis, ranging in complexity, modelling assumptions (i.e. what the underlying mathematics assumes about the data), uses and computational demand. Nonetheless, the underlying common theme of most regression models is that they are a way of representing data in a mathematical way to draw meaning/ inference about potential relationships and associations. 

Clearly, the notion that one can precisely describe complex health or biological processes in terms of mathematical formulae might seem implausible. However, the aim of the model is to reflect aspects of the systems/ data in a sufficiently approximate way, so as to draw valuable meaning. The statistician George Box famously phrased this as "all models are wrong, but some are useful". 

But given the vast range of regression methods that are avaliable, how can we choose which one is appropriate? To a large extent this depends on the structure of the data and the types of outcomes. For example, if your data contains repeated measures (as in the full Framingham dataset, where an individual was measured at multiple time points), then there are special types of regression models that can handle this: i.e. longitudinal data analysis. 

Alternatively, the type of outcome can also play a role in choosing the analysis. While there are some exceptions, the below gives a general rule-of-thumb:

- Continuous outcome: generally, we would use a linear regression model (think of this as a 'line of best fit' through the scatter plot, for example)

- Binary (yes/no) categorical outcome: generally, we would use a special type of regression model called logistic regression

- Count outcome: here, we would normally use a type of modelling called Poisson regression

In most medical applications, continuous and binary outcomes are common. 

So, lets consider how we can apply this to our hypothetical example. 

__Activity: Based on the fact that we are interested in exploring the association between SBP and other variables, then our outcome is SBP. Spend a few moments thinking about what type of outcome variable this is (i.e. continuous, binary, count, categorical, etc.), and therefore what type of regression you might apply. What about if we were instead interested in modelling cardiovascular disease?__

In the SBP case, this is a continuous outcome, so we might choose linear regression. Lets use R to see what types of interpretation we can draw from such models. To fit a linear regression model in R, we make use of the 'lm()' function:

In [ ]:
SBP_model <- lm(SYSBP ~ 1 + BMI, data = frmgham_1visit_data) 
#again, dont worry too much about the code here: you can think of this as SYSBP = intercept + slope*BMI

In this case, we are regressing one variable (BMI) against systolic blood pressure. I think the most intuitive way of understanding the output/interpretation of this model is to superimpose the above "line of best fit" (i.e. the linear regression) onto our previous scatter plot using the following code: 

In [ ]:
Line_of_Best_Fit <- predict(SBP_model) #extracts what we call the fitted values from the model
plot(x = frmgham_1visit_data$BMI, y = frmgham_1visit_data$SYSBP, xlab = "BMI", ylab = "SBP")
lines(x = frmgham_1visit_data$BMI[which(!is.na(frmgham_1visit_data$BMI))], #handling of missing BMI in the data 
      y = as.vector(Line_of_Best_Fit), col="red") #superimposes the fitted values onto the scatter

From this we can see that the line goes "up" for higher values of BMI. This agrees with our earlier observation of a positive correlation. The model can reveal the strength of this association - i.e. the gradient of the slope. We can extract this as follows:

In [ ]:
coefficients(SBP_model)

We interpret these numbers as follows: the expected SBP for someone with a BMI=0 is 86.67 (intercept); the model expects SBP to increase by 1.79 for each unit increase in BMI (the gradient). 

Understanding all of the outputs from regression models is beyond what we need to cover in this module. Suffice to say that we can use regression analysis to explore associations. 

Equally, we can easily use such regression models to make predictions: for example, if we wanted to use the above model to predict what we expect the SBP to be for someone with a BMI of 30, we could trace up from 30 on the x-axis and read across to the y-axis at the red line. 

The real strength of regression modelling is the ability to model the relationship between multiple variables and an outcome. If we are interested in the relationship between two or more variables, then we can simply add them into the regression model. So-called multiple linear regression models are harder to visualise than the above case (since we move outside of 2D space), but the same ideas and (more importantly for this module) interpretation applies. 

In summary, regression models take the form of: outcome = deterministic model + residual error. We interpret this as meaning that the model describes the data as the sum of a systematic mathematical form plus some random contribution that cannot yet be explained: you might hear this described as signal and noise. 

__Activity:__ 

1) You might wish to use the empty code cell below to explore these data further using linear regression, by modifying the above code. Try to add age into the model, and try to see if you can interpret the output.

2) Begin to consider the type of "problem" and "plan" for your DTP case study. Are any of the questions related to exploring associations/predictions? If so, consider which types of regression might be appropriate.

In [ ]:
##You can write R code here, then click the "Run" button in the toolbar at the top to execute your code.

## Correlation doesnt mean causation
Our final "type" of statistical analysis - i.e. causation - usually requires very careful study design and statistical methods. The term causation means that observing a change in one event/variable directly causes or is responsible for a change in another event/variable. 

For example, it is now widely believed that smoking causes lung cancer, in the sense that the act of smoking causes a change in the cells of the lungs, which in turn causes lung cancer. 

Turning data into evidence of a causal effect requires very careful study design, often far beyond what is required in a association/prediction "type" of statistical analysis. 

To illustrate this, there are many humours examples of spurious correlations/associations, which dont necessarily mean causation (see, for example, http://www.tylervigen.com/spurious-correlations). 

As a more health-related illustration, lets return to our hypothetical example from above, where we fitted a model to predict SBP using BMI. Here, we found a positive association between BMI and SBP. But, would it be fair/accurate to say that high BMI causes higher SBP, based on the data/ information we have here? Maybe it does, but maybe it doesnt - we simply can't say for sure based on the above data. Perhaps the causation is actually the other way around: high SBP causes high BMI. Or perhaps there is another variable, that increases both BMI and SBP: such a variable is called an confounder. All of these uncertainties drive the common phrase: "correlation does not mean causation".

Study designs such as randomised controlled trials (RCTs) are often seen as the pillar of providing causal estimates since the randomisation process aims to remove confounding. 

However, other designs are sometimes possible, and observational data can be used to move towards causal evidence. Take John Snow and the Broad Street water pump as the classic example of this (https://en.wikipedia.org/wiki/John_Snow). There was certainly no active process of randomising one group of participants to drink from the Broad Street pump and another group to drink from another independent pump! Instead, John Snow essentially used a natural experiment and based his findings from observational data. 

There are statistical methods that aim to allow one to draw causal inference from observational data (interested readers might consider analyses such as propensity scores, for example). Additionally, prospective matched-case-control study designs can also be useful, to some extent, in moving towards causal claims (albeit not themselves a solution per se).

The point is that, causal inference requires careful study design, and needs to consider the likely causal pathways, usually from a clinical perspective. 

__Activity: Spend some time, at this stage, thinking about what type of "plan" (in terms of our design cycle) would be needed to make any causal claims/evidence within your DTP case studies. If such causal evidence isn't directly relevant for your case study, then think about what future related-research (i.e. other problems) might be interested in causal evidence.__

## Summary of the workshop
In this workshop, we have explored the use of regression models as a way of investigating associations between multiple variables and an outcome-of-interest. Such models provide a mathematical representation of the associations, whereby the coefficients (i.e. gradients) indicate how much the model expects the outcome to change for unit changes in the variables. At design stage, the type of outcome under consideration often drives the type of regression model that one would consider in the analysis plans. Finally, we need cautious interpretation of such models: correlation does not imply causation. 

The overall message of this workshop (and the last one from week 3) is that the design cycle of (i) problem, (ii) plan, (iii) data, (iv) analysis, and (v) conclusion, is pivotal in turning data into evidence. This can be achieved by viewing statistical design through a problem-centric approach. As health informaticians, your role is not necessarily to produce the statistics, but be able to consume them in such a way as to know what "types" of statistical analysis are appropriate for investigating different types of "problem". After which, it is the task of communicating the statistical results to different stakeholders in order to drive evidence-based change, through appropriate intervention.

If you wish to explore these concepts further, the "Advanced (Optional) Activity" below might be of interest in exploring a multivariable logistic regression model.

## Advanced (Optional) Activity
In all of the regression models above, we were interested in describing the association between variables and a continuous outcome. Therefore, we opted to fit a linear regression model. However, in many health contexts, the outcome will be binary (e.g. yes/no mortality after surgery). In this case fitting linear regression models would not be appropriate, and instead we turn to logistic regression (as mentioned in the lectures for this week). 

To explore logistic regression, lets again use the Framingham dataset, but imagine that our problem statement is that we wish to investigate the association between age, SBP, and BMI on the risk of developing cardiovascular disease (CVD). Here, our outcome (CVD) is binary (coded as 0 or 1 in our dataset), so we can fit a logistic regression model directly to our data. 

For illustration purposes, lets start by just considering one variable (e.g. age). To do this in R, we make use of the glm() function (stands for generalised linear model), and set the "family" option within this function to be "binomial" (dont worry about this: its simply telling R to fit a logistic regression model):

In [ ]:
CVD_model <- glm(CVD ~ 1 + AGE, data = frmgham_1visit_data, family = binomial) 
coefficients(CVD_model) #return the estimated coefficients from the logistic regression model

How do we interpret these numbers? Our main interest is in the AGE coefficient: in the case of a logistic regression model the above numbers are what we call log-odds. Any log-odd value above 0 means that the corresponding variable increases the risk (or odds) of the outcome occuring, while any log-odd value below 0 mens that the corresponding variable decreases the risk (or odds) of the outcome occuring. So in the above example, we see that the log-odds of age is above 0, so we conclude that higher age is associated with high risk of CVD. _Note that we will not be exploring statistical significance in this course, but one would need to test if this is a significant increase (in others words, is this log-odds number significantly different from 0, which would imply no association?). For those interested, you could look at running the R code: summary(CVD_model) and examining the p-value (given below the "Pr(>|t|)" output); SPOILER: doing so shows that age significantly increases the risk of CVD._

Oftentimes, we want to explore multiple associations within the single model. This allows us to "adjust for" potential confounders, or explore how multiple variables potentially associate with the outcome of interest. Lets see an example of this when looking at CVD. In R, we still use the glm() command, but simply add all of the variables we are interested in. Recall from above that we are interested in age, SBP, and BMI on risk of CVD. In R, we have:

In [ ]:
CVD_model2 <- glm(CVD ~ 1 + AGE + SYSBP + BMI, data = frmgham_1visit_data, family = binomial) 
coefficients(CVD_model2) #return the estimated coefficients from the logistic regression model

Notice that we now get multiple coefficients: one for each of age, SYSBP and BMI. The interpretation remains similar to earlier: log-odds above 0 suggest a increased risk of the outcome (in this case CVD), and _vice versa_. In this example, we see that all three variables associate positively with the risk of CVD in the sense that increases in any of them increase the risk of CVD. The important distinction is that they are each "adjusted for" all of the other variables in the model. For example, notice how the log-odds for age is slightly lower (in absolute terms) than it was from the univariable model we fitted a moment ago. This is because this log-odds is now 'adjusted for' SYSBP and BMI (in the sense that we now have a log-odds estimate of the association between age and risk of CVD, while holding all other variables constant - and similarly for the other variables: e.g. log-odds of BMI, holding age and SYSBP contant). 

__Note: In all of the above interpretation we have avoided any discussion of statistical significance of these results. In practice one needs to consider this (e.g. is the association significant?) through appropriate tests. This is a topic in itself and will be not considered in this course. As a very brief example for those interested, all of the above log-odds from our last model are statistically significant, which you can explore through p-values from running summary(CVD_model2). Again, this is for interest, and we dont go into this detail in this module.__

In [ ]:
##You can write R code here, then click the "Run" button in the toolbar at the top to execute your code.